In [78]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import confusion_matrix, auc, roc_curve

In [25]:
# data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
model_summary = FastText.load("models-4.1(a)/model_summary.bin")


In [118]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
data.columns =[column.replace(" ", "_") for column in data.columns] 
ls = [1 if i < 4761 else 0 for i in range(47611)]
# data.set_index("Like Count", inplace=True)
data.sort_values("Like_Count", inplace=True,ascending=False)
data['class'] = ls
data.query(' Post_Type == "photo" or Post_Type == "video" or Post_Type == "link" ', inplace=True)
data.reset_index(inplace=True, drop=True)
data = data.truncate(before=0,after=4761*2)
data = data.sample(frac=1)


In [1]:
post_types = set(data["Post_Type"].tolist())
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in post_types:
    cat_vec[c] = model_summary[c]    
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Post_Type"][h]]))
    head_vec.append(sum1)

NameError: name 'data' is not defined

In [120]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = data["class"][:feat_len].tolist()
test = head_vec[feat_len::]
res_test = data["class"][feat_len:].tolist()
print(len(train), len(res_train))

7618 7618


In [121]:
model_svm_50100 = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [122]:
y = svmutil.svm_predict(res_test, test, model_svm_50100)

Accuracy = 55.3806% (1055/1905) (classification)


In [123]:
y[0]

[0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0

In [124]:
confusion_matrix(res_test, y[0], labels=[0,1])

array([[592, 384],
       [466, 463]])

In [125]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [126]:
recall = tp/(tp+fn)
recall

0.4983853606027987

In [127]:
precision = tp/(tp+fp)
precision

0.5466351829988194

In [128]:
f1 = precision*recall*2/(precision+recall)
f1

0.5213963963963965

In [130]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

0.5524713688259895

In [131]:
svmutil.svm_save_model("model_b_svm.bin",model_svm_50100)

In [132]:
svmutil.svm_train(res_train, train,'-v 10 -t 2 -c 2')

Cross Validation Accuracy = 55.697%


55.69703334208454

In [77]:
# post_types

In [76]:
# data2 = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
# post_types2 = set(data2["Post Type"].tolist())
# post_types2

In [75]:
# dic = {'event':0, 'link':0, 'photo':0, 'status':0, 'video':0}
# posts = data["Post_Type"].tolist()
# for p in posts:
#     dic[p]+=1
# dic

In [117]:
dic = {0:0, 1:0}
posts = y[0]
for p in posts:
    dic[p]+=1
dic

{0: 2857, 1: 0}